In [1]:
from dataclasses import dataclass
import datetime
from germansentiment import SentimentModel
import html
import pandas as pd 
import re
import snscrape.modules.twitter as sntwitter
import sqlite3
import torch

In [2]:
pd.set_option('display.max_colwidth', None)
db_file = 'tweets-de.db'

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

In [4]:
@dataclass
class Query:
    search_term: str
    lang: str
    date_from: datetime.datetime
    date_to: datetime.datetime
    period: str = "Unspecified"

    def __post_init__(self):
        if (self.period == "Unspecified"):
            self.period = self.date_from.strftime('%Y') + '-' + self.date_to.strftime('%Y')

    def __str__(self):
        return(self.search_term  + " lang:" + self.lang + " until:" + self.date_to.strftime('%Y-%m-%d') + " since:" + self.date_from.strftime('%Y-%m-%d'))

In [5]:
def preprocess_tweet(tweet):
    tweet.rawContent = html.unescape(tweet.rawContent)
    tweet.rawContent = re.sub("@[A-Za-z0-9_]+", "", tweet.rawContent).strip()
    tweet.rawContent = re.sub("#", "", tweet.rawContent).strip()
    tweet.rawContent = re.sub(r"http\S+", "", tweet.rawContent)
    tweet.rawContent = re.sub("(\\n)+", " ", tweet.rawContent)
    tweet.rawContent = re.sub("(\\'er)", "er", tweet.rawContent)
    if tweet.hashtags is not None:
        tweet.hashtags = ";".join(tweet.hashtags)
    return tweet

In [6]:
def gather_tweets_to_sql(query, conn, limit = 100):
    sql_insert = ''' INSERT INTO tweets(id, date, user, lang, search_term, period, content, hashtags, sentiment)
              VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?) '''
    i = 0
    for tweet in sntwitter.TwitterSearchScraper(str(query)).get_items():
        if i == limit:
            break
        else:
            i += 1
            tweet = preprocess_tweet(tweet)
            sql_tweet= (tweet.id, tweet.date, tweet.user.username, tweet.lang, query.search_term, query.period, tweet.rawContent, tweet.hashtags, None)
            cur = conn.cursor()
            cur.execute(sql_insert, sql_tweet)
            conn.commit()

In [7]:
def recode_sentiment(sentiment):
    if sentiment == "positive":
        return 1
    elif sentiment == "negative":
        return -1
    else:
        return 0

In [8]:
sql_create_tweets_table = """CREATE TABLE tweets (
                                       id integer NOT NULL,
                                       date TIMESTAMP NOT NULL,
                                       user TEXT NOT NULL,
                                       lang TEXT NOT NULL,
                                       search_term TEXT NOT NULL,
                                       period TEXT NOT NULL,
                                       content TEXT NOT NULL,
                                       hashtags TEXT,
                                       sentiment integer,
                                       PRIMARY KEY (id, search_term));"""

In [286]:
# create a database connection
conn = create_connection(db_file)
create_table(conn, sql_create_tweets_table)
conn.close()

<function Connection.close()>

In [288]:
search_terms = ["Berlin", "Hamburg", "Köln", "München", "Frankfurt"]
years_from = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
limit = 10000
conn = create_connection(db_file)
for s in search_terms:
    for y in years_from:
        q = Query(s, "de", datetime.datetime(y, 12, 31), datetime.datetime(y+1, 1, 2))
        gather_tweets_to_sql(q, conn, limit)
conn.close()

<function Connection.close()>

In [9]:
model = SentimentModel()

In [19]:
sql_select = "SELECT id, content FROM tweets ORDER BY id LIMIT ? OFFSET ?"
sql_update = "UPDATE tweets SET sentiment = ? WHERE id = ?"
conn = create_connection(db_file)
cur = conn.cursor()

cur.execute("SELECT COUNT(*) from tweets")
n_rows = cur.fetchone()[0]
batch_size = 1000
offset = 0
while True:
    cur.execute(sql_select, (batch_size, offset))
    batch = pd.DataFrame(list(cur), columns = ['id', 'content'])
    batch['sentiment'] = list(map(recode_sentiment, model.predict_sentiment(batch['content'])))
    batch = batch[['sentiment', 'id']].values.tolist()
    cur.executemany(sql_update, batch)
    offset += batch_size
    if (offset >= n_rows):
        break
conn.commit()
conn.close()


In [20]:
del model
torch.cuda.empty_cache()